# Modified from https://www.kaggle.com/code/jtrotman/winning-team-submission-traces/notebook

In [1]:
import gc, os, sys, time
import pandas as pd, numpy as np
from unidecode import unidecode
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from IPython.display import HTML, display

### Original Code

In [2]:
# CSV_DIR = Path('..', 'input', 'meta-kaggle')
CSV_DIR = Path('D:/Academics/Research/IORA/Game Designer/Data Files/', 'input', 'meta-kaggle')
if not CSV_DIR.is_dir():
    CSV_DIR = Path('D:/Academics/Research/IORA/Game Designer/Data Files/', 'input')

def read_csv_filtered(csv, col, values):
    dfs = [df.loc[df[col].isin(values)]
           for df in pd.read_csv(CSV_DIR / csv, chunksize=100000, low_memory=False)]
    return pd.concat(dfs, axis=0)

comps = pd.read_csv(CSV_DIR / 'Competitions.csv').set_index('Id')
comps = comps.query("HostSegmentTitle != 'InClass'")
idx = comps.EvaluationAlgorithmName.isnull()
comps.loc[idx, 'EvaluationAlgorithmName'] = comps.loc[idx, 'EvaluationAlgorithmAbbreviation']

comps['EvaluationLabel'] = comps.EvaluationAlgorithmAbbreviation
idx = comps.EvaluationLabel.str.len() > 30
comps.loc[idx, 'EvaluationLabel'] = comps.loc[idx, 'EvaluationLabel'].str.replace(r'[^A-Z\d\-]', '', regex=True)

comps['DeadlineDate'] = pd.to_datetime(comps.DeadlineDate)
comps['EnabledDate'] = pd.to_datetime(comps.EnabledDate)
comps['DeadlineDateText'] = comps.DeadlineDate.dt.strftime('%c')
comps['EnabledDateText'] = comps.EnabledDate.dt.strftime('%c')
comps['Year'] = comps.DeadlineDate.dt.year
comps['RewardQuantity'].fillna('', inplace=True)
comps['Days'] = (comps.DeadlineDate - comps.EnabledDate) / pd.Timedelta(1, 'd')
comps['FinalWeek'] = (comps.DeadlineDate - pd.Timedelta(1, 'w'))

teams = read_csv_filtered('Teams.csv', 'CompetitionId', comps.index).set_index('Id')
# Just the winning teams (Change to different settings)
# teams = teams.query('PrivateLeaderboardRank==1').copy()
teams = teams.query('PrivateLeaderboardRank<=20').copy()

tmemb = read_csv_filtered('TeamMemberships.csv', 'TeamId', teams.index).set_index('Id')
users = read_csv_filtered('Users.csv', 'Id', tmemb.UserId)
tmemb = tmemb.merge(users, left_on='UserId', right_on='Id')

# Submissions
subs = read_csv_filtered('Submissions.csv', 'TeamId', tmemb.TeamId)
subs = subs.rename(columns={'PublicScoreFullPrecision': 'Public'})
subs = subs.rename(columns={'PrivateScoreFullPrecision': 'Private'})
subs['SubmissionDate'] = pd.to_datetime(subs.SubmissionDate)

asfloats = ['PublicScoreLeaderboardDisplay',
            'Public',
            'PrivateScoreLeaderboardDisplay',
            'Private',]

subs[asfloats] = subs[asfloats].astype(float)
# subs.IsAfterDeadline.mean()

subs = subs.query('not IsAfterDeadline').copy()
subs['CompetitionId'] = subs.TeamId.map(teams.CompetitionId)
subs['CompetitionSlug'] = subs.CompetitionId.map(comps.Slug)
subs['TeamName'] = subs.TeamId.map(teams.TeamName)

# subs['CompetitionId'].nunique()

# values some competitions use as invalid scores
for bad in [99, 999999]:
    for c in asfloats:
        idx = (subs[c] == bad)
        subs.loc[idx, c] = subs.loc[idx, c].replace({bad: np.nan})

# Display order: most recent competitions first
subs = subs.sort_values(['CompetitionId', 'Id'], ascending=[False, True])

### Modified Code
First specify the list of competitions by their slugs -- comp_list. The returned **subs** contain all submission info for the specified competitions

In [2]:
# Change the list of competitions of our choice
comp_list = ['vinbigdata-chest-xray-abnormalities-detection', # Medical Image
             'talkingdata-adtracking-fraud-detection', # NLP Text Fraud
             'landmark-recognition-2021', # Big Tech CV
             'chaii-hindi-and-tamil-question-answering', # NLP Q&A,
             'nfl-health-and-safety-helmet-assignment'] # Sports CV

In [3]:
# CSV_DIR = Path('..', 'input', 'meta-kaggle')
CSV_DIR = Path('D:/Academics/Research/IORA/Game Designer/Data Files/', 'input', 'meta-kaggle')
if not CSV_DIR.is_dir():
    CSV_DIR = Path('D:/Academics/Research/IORA/Game Designer/Data Files/', 'input')

def read_csv_filtered(csv, col, values):
    dfs = [df.loc[df[col].isin(values)]
           for df in pd.read_csv(CSV_DIR / csv, chunksize=100000, low_memory=False)]
    return pd.concat(dfs, axis=0)

comps = pd.read_csv(CSV_DIR / 'Competitions.csv').set_index('Id')
comps = comps.query("HostSegmentTitle != 'InClass'")
idx = comps.EvaluationAlgorithmName.isnull()
comps.loc[idx, 'EvaluationAlgorithmName'] = comps.loc[idx, 'EvaluationAlgorithmAbbreviation']

comps['EvaluationLabel'] = comps.EvaluationAlgorithmAbbreviation
idx = comps.EvaluationLabel.str.len() > 30
comps.loc[idx, 'EvaluationLabel'] = comps.loc[idx, 'EvaluationLabel'].str.replace(r'[^A-Z\d\-]', '', regex=True)

comps['DeadlineDate'] = pd.to_datetime(comps.DeadlineDate)
comps['EnabledDate'] = pd.to_datetime(comps.EnabledDate)
comps['DeadlineDateText'] = comps.DeadlineDate.dt.strftime('%c')
comps['EnabledDateText'] = comps.EnabledDate.dt.strftime('%c')
comps['Year'] = comps.DeadlineDate.dt.year
comps['RewardQuantity'].fillna('', inplace=True)
comps['Days'] = (comps.DeadlineDate - comps.EnabledDate) / pd.Timedelta(1, 'd')
comps['FinalWeek'] = (comps.DeadlineDate - pd.Timedelta(1, 'w'))
comp_id = comps[comps['Slug'].isin(comp_list)].index # Competition ID of our choice

teams = read_csv_filtered('Teams.csv', 'CompetitionId', comp_id).set_index('Id') # Teams that participate in these competitions
tmemb = read_csv_filtered('TeamMemberships.csv', 'TeamId', teams.index).set_index('Id')
users = read_csv_filtered('Users.csv', 'Id', tmemb.UserId)
tmemb = tmemb.merge(users, left_on='UserId', right_on='Id') # Some further cleaning of teams

# Submissions
subs = read_csv_filtered('Submissions.csv', 'TeamId', tmemb.TeamId) # Submission of these teams
subs = subs.rename(columns={'PublicScoreFullPrecision': 'Public'})
subs = subs.rename(columns={'PrivateScoreFullPrecision': 'Private'})
subs['SubmissionDate'] = pd.to_datetime(subs.SubmissionDate)

asfloats = ['PublicScoreLeaderboardDisplay',
            'Public',
            'PrivateScoreLeaderboardDisplay',
            'Private',]

subs[asfloats] = subs[asfloats].astype(float)

subs = subs.query('not IsAfterDeadline').copy()
subs['CompetitionId'] = subs.TeamId.map(teams.CompetitionId)
subs['CompetitionSlug'] = subs.CompetitionId.map(comps.Slug)
subs['TeamName'] = subs.TeamId.map(teams.TeamName)

# values some competitions use as invalid scores
for bad in [99, 999999]:
    for c in asfloats:
        idx = (subs[c] == bad)
        subs.loc[idx, c] = subs.loc[idx, c].replace({bad: np.nan})

# Display order: most recent competitions first
subs = subs.sort_values(['CompetitionId', 'Id'], ascending=[False, True])

In [4]:
teams.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61269 entries, 1370425 to 8629666
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CompetitionId                   61269 non-null  int64  
 1   TeamLeaderId                    61260 non-null  float64
 2   TeamName                        61266 non-null  object 
 3   ScoreFirstSubmittedDate         0 non-null      float64
 4   LastSubmissionDate              7410 non-null   object 
 5   PublicLeaderboardSubmissionId   7372 non-null   float64
 6   PrivateLeaderboardSubmissionId  7372 non-null   float64
 7   IsBenchmark                     61269 non-null  bool   
 8   Medal                           841 non-null    float64
 9   MedalAwardDate                  844 non-null    object 
 10  PublicLeaderboardRank           7369 non-null   float64
 11  PrivateLeaderboardRank          7369 non-null   float64
dtypes: bool(1), float64(7), 

In [5]:
subs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136833 entries, 9561079 to 3964610
Data columns (total 14 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Id                              136833 non-null  int64         
 1   SubmittedUserId                 136833 non-null  float64       
 2   TeamId                          136833 non-null  int64         
 3   SourceKernelVersionId           62245 non-null   float64       
 4   SubmissionDate                  136833 non-null  datetime64[ns]
 5   ScoreDate                       136583 non-null  object        
 6   IsAfterDeadline                 136833 non-null  bool          
 7   PublicScoreLeaderboardDisplay   129946 non-null  float64       
 8   Public                          129946 non-null  float64       
 9   PrivateScoreLeaderboardDisplay  129946 non-null  float64       
 10  Private                         129946 non-null  

### This chunk is to save data (globally) that 
1. summarize relevant information of respective competitions (**XXX_summary**), and 
2. filter the teams that satisfy our criteria (**XXX_filtered**)

In [8]:
useful_cols = ['SubmissionDate',
               'Public',
               'Private',
               'TeamId',
               'TeamName']

for c in comp_list:
    cname = c.split('-')[0]
    df = globals()[cname] = subs.query(f"CompetitionSlug=='{c}'") # Raw DataFrame for the Competition
    
    key_col = 'TeamId'
    last_day = df['SubmissionDate'].max()

    names = df[['TeamId', 'TeamName']].drop_duplicates().set_index('TeamId')
    score = df[df['SubmissionDate'] == last_day].groupby(key_col)['Private'].max().sort_values(ascending=False)

    submission = df.groupby(key_col)['SubmissionDate'].count().sort_values(ascending=False)
    submission.name = 'Submissions'

    first_day = df.groupby(key_col)['SubmissionDate'].min()
    first_day.name = 'FirstDay'

    last_day = df.groupby(key_col)['SubmissionDate'].max()
    last_day.name = 'LastDay'

    duration = last_day - first_day
    duration.name = 'Duration'

    summary = pd.concat([names, score, duration, first_day, last_day, submission],axis=1).sort_values(['Private', 'Duration', 'Submissions'], ascending=False)
    summary = summary.dropna(axis=0)
    
    summary_name = cname + '_summary'
    globals()[summary_name] = summary # Summary of Rankings for the Competition
    # print(f'{summary_name} is saved')
    
    filtered_team = summary.index[:2] # Need Modification for our criteria
    filtered_data = df[df['TeamId'].isin(filtered_team)][useful_cols]
    start = filtered_data.groupby('TeamId')['SubmissionDate'].min().max()
    # filtered_data = filtered_data[filtered_data['SubmissionDate'] >= later_enter_date]
    
    filtered_name = cname + '_filtered'
    globals()[filtered_name] = filtered_data
    # print(f'{filtered_name} is saved')

    filtered_data['t'] = (filtered_data.SubmissionDate - start).dt.days // 7 + 1
    # print(filtered_data)
    q_x = filtered_data.groupby(by=['TeamId', 't']).agg(q =pd.NamedAgg('SubmissionDate', aggfunc='count'),
                                       x = pd.NamedAgg('Private', aggfunc='max'),
                                       public = pd.NamedAgg('Public', aggfunc='max'))
    df_i = q_x.loc[filtered_team[0]]
    df_j = q_x.loc[filtered_team[1]]
    df_full = df_i.merge(df_j,left_index=True,right_index=True,how='outer', suffixes=('_i','_j'))
    df_full[['q_i', 'q_j']] = df_full[['q_i', 'q_j']].fillna(0)
    df_full[['x_i', 'x_j', 'public_i', 'public_j']] = df_full[['x_i', 'x_j', 'public_i', 'public_j']].fillna(method='ffill')
    df_full['Z'] = df_full['public_i'] - df_full['public_j']
    cleaned_data = df_full[df_full.index > 0]

    cleaned_name = cname + '_cleaned'
    globals()[cleaned_name] = cleaned_data
    cleaned_data.to_csv(cleaned_name + '.csv', index=False)
    print(f'{cleaned_name} is saved')

vinbigdata_cleaned is saved
talkingdata_cleaned is saved
landmark_cleaned is saved
chaii_cleaned is saved
nfl_cleaned is saved


### Summary Saved

In [9]:
nfl_summary.head()

,TeamName,Private,Duration,FirstDay,LastDay,Submissions
TeamId,,,,,,
7279260,K_mat,0.951347,69 days,2021-08-25,2021-11-02,49
7243287,tito,0.910460,80 days,2021-08-14,2021-11-02,50
7245967,fantastic_hirarin,0.904170,65 days,2021-08-29,2021-11-02,26
7244891,NFL 2nd half,0.888086,74 days,2021-08-20,2021-11-02,98
7300455,KL,0.885374,59 days,2021-09-04,2021-11-02,76


In [10]:
chaii_summary.head()

,TeamName,Private,Duration,FirstDay,LastDay,Submissions
TeamId,,,,,,
7251121,Hold my chai ☕️,0.792470,96 days,2021-08-11,2021-11-15,198
7252769,tkm kh,0.785359,95 days,2021-08-12,2021-11-15,259
7251967,Magic,0.777883,95 days,2021-08-12,2021-11-15,121
7253814,"Zacchaeus, zzy",0.777256,95 days,2021-08-12,2021-11-15,302
7250918,Abhishek + Udbhav,0.776591,96 days,2021-08-11,2021-11-15,253


In [11]:
vinbigdata_summary.head()

,TeamName,Private,Duration,FirstDay,LastDay,Submissions
TeamId,,,,,,
6087561,SZI,0.311226,85 days,2021-01-04,2021-03-30,381
6090014,scumed,0.305267,81 days,2021-01-08,2021-03-30,448
6129731,fantastic_hirarin,0.300330,71 days,2021-01-18,2021-03-30,68
6087654,Kiet Chu,0.300266,78 days,2021-01-11,2021-03-30,140
6311035,Watercooled,0.299402,36 days,2021-02-22,2021-03-30,128


In [12]:
landmark_summary.head()

,TeamName,Private,Duration,FirstDay,LastDay,Submissions
TeamId,,,,,,
7252683,Dieter,0.512726,45 days,2021-08-17,2021-10-01,83
7252015,bestfitting,0.494300,45 days,2021-08-17,2021-10-01,122
7256588,ByteDance Vision Tech.,0.489626,37 days,2021-08-25,2021-10-01,212
7282910,NotEnoughFitting,0.483593,38 days,2021-08-24,2021-10-01,135
7312380,All Data Are Ext,0.483143,42 days,2021-08-20,2021-10-01,117


In [13]:
talkingdata_summary.head()

,TeamName,Private,Duration,FirstDay,LastDay,Submissions
TeamId,,,,,,
1479247,"['flowlight', 'komaki'].shuffle()",0.984332,46 days,2018-03-22,2018-05-07,106
1462555,PPP is already in use,0.984126,57 days,2018-03-11,2018-05-07,146
1449364,bestfitting,0.984084,37 days,2018-03-31,2018-05-07,173
1512459,K.A.C.,0.984027,60 days,2018-03-08,2018-05-07,219
1485607,MMDP,0.984019,60 days,2018-03-08,2018-05-07,156


In [11]:
ranzcr_filtered

,SubmissionDate,Public,Private,TeamId,TeamName
8018725,2020-12-16,0.862191,0.870162,6012213,リーマン面を這う
8018726,2020-12-16,0.861558,0.868040,6012213,リーマン面を這う
8018844,2020-12-18,0.927493,0.935666,6012213,リーマン面を這う
8018845,2020-12-18,0.895836,0.898207,6012213,リーマン面を這う
8018721,2020-12-18,0.904338,0.909976,6012213,リーマン面を這う
...,...,...,...,...,...
8751234,2021-03-16,0.975586,0.976732,6013849,All Data Are Ext
8751235,2021-03-16,0.975616,0.976657,6013849,All Data Are Ext
8750940,2021-03-16,0.974120,0.976420,6012213,リーマン面を這う
8750941,2021-03-16,0.973660,0.975713,6012213,リーマン面を這う


In [12]:
vinbigdata_filtered

,SubmissionDate,Public,Private,TeamId,TeamName
8183501,2021-01-04,0.164498,0.159467,6087561,SZI
8183502,2021-01-06,0.124063,0.146764,6087561,SZI
8183503,2021-01-06,0.117363,0.133969,6087561,SZI
8380861,2021-01-07,0.172138,0.167738,6087561,SZI
8380862,2021-01-07,0.160037,0.164868,6087561,SZI
...,...,...,...,...,...
8705991,2021-03-30,0.316657,0.291157,6090014,scumed
8628181,2021-03-30,0.296349,0.305256,6087561,SZI
8628179,2021-03-30,0.287484,0.299893,6087561,SZI
8628183,2021-03-30,0.296358,0.304209,6087561,SZI


In [13]:
avito_filtered

,SubmissionDate,Public,Private,TeamId,TeamName
3740838,2018-04-26,0.283540,0.286429,1627465,eprst
3740839,2018-04-26,0.236018,0.239847,1627465,eprst
3741243,2018-04-26,0.236021,0.239831,1627465,eprst
3740837,2018-04-26,0.236192,0.240035,1627465,eprst
3740840,2018-04-26,0.318184,0.320441,1627465,eprst
...,...,...,...,...,...
3743621,2018-06-27,0.769680,0.765664,1781096,Charles Pieri
3741250,2018-06-27,0.215835,0.219503,1627465,eprst
3741255,2018-06-27,0.225301,0.229674,1627465,eprst
3741256,2018-06-27,0.226653,0.229937,1627465,eprst


In [14]:
talkingdata_filtered

,SubmissionDate,Public,Private,TeamId,TeamName
3428010,2018-03-11,0.963049,0.962580,1462555,PPP is already in use
3428005,2018-03-11,0.954556,0.955195,1462555,PPP is already in use
3428006,2018-03-13,0.960498,0.962559,1462555,PPP is already in use
3428007,2018-03-13,0.964039,0.963028,1462555,PPP is already in use
3428008,2018-03-13,0.962027,0.963548,1462555,PPP is already in use
...,...,...,...,...,...
3968958,2018-05-07,0.983438,0.984322,1479247,"['flowlight', 'komaki'].shuffle()"
3923700,2018-05-07,0.983309,0.984113,1462555,PPP is already in use
3923701,2018-05-07,0.983317,0.984126,1462555,PPP is already in use
3923698,2018-05-07,0.983334,0.984014,1462555,PPP is already in use


### Cleaned Data

In [14]:
nfl_cleaned

,q_i,x_i,public_i,q_j,x_j,public_j,Z
t,,,,,,,
1,5.0,0.668100,0.743317,0.0,0.629647,0.710539,0.032778
2,2.0,0.840817,0.847551,0.0,0.629647,0.710539,0.137011
3,4.0,0.904971,0.879856,0.0,0.629647,0.710539,0.169317
4,6.0,0.894157,0.888637,0.0,0.629647,0.710539,0.178098
6,1.0,0.886581,0.923123,0.0,0.629647,0.710539,0.212584
7,0.0,0.886581,0.923123,10.0,0.876757,0.860337,0.062786
8,8.0,0.930886,0.932258,0.0,0.876757,0.860337,0.071921
9,5.0,0.938919,0.929044,5.0,0.905786,0.924171,0.004873
10,18.0,0.956714,0.939928,20.0,0.915345,0.935250,0.004679


In [15]:
chaii_cleaned

,q_i,x_i,public_i,q_j,x_j,public_j,Z
t,,,,,,,
1,4.0,0.706188,0.735128,37.0,0.744709,0.775963,-0.040836
2,0.0,0.706188,0.735128,25.0,0.752344,0.778851,-0.043723
3,0.0,0.706188,0.735128,29.0,0.750668,0.786423,-0.051295
4,0.0,0.706188,0.735128,17.0,0.751640,0.787606,-0.052478
5,0.0,0.706188,0.735128,26.0,0.775477,0.816295,-0.081167
6,0.0,0.706188,0.735128,25.0,0.775910,0.821692,-0.086564
7,0.0,0.706188,0.735128,13.0,0.772680,0.820828,-0.085701
8,6.0,0.702023,0.728130,15.0,0.775624,0.827030,-0.098900
9,24.0,0.729355,0.761345,3.0,0.776146,0.821473,-0.060128


In [16]:
vinbigdata_cleaned

,q_i,x_i,public_i,q_j,x_j,public_j,Z
t,,,,,,,
1,6,0.186076,0.168659,25.0,0.203942,0.204077,-0.035418
2,31,0.259240,0.245043,50.0,0.229350,0.257519,-0.012476
3,44,0.286489,0.284101,33.0,0.263297,0.266068,0.018033
4,43,0.289577,0.300183,63.0,0.272836,0.280229,0.019955
5,30,0.281129,0.293261,53.0,0.265559,0.278989,0.014272
6,29,0.297532,0.301950,35.0,0.233009,0.279254,0.022695
7,34,0.301086,0.309335,45.0,0.264398,0.281621,0.027714
8,32,0.309336,0.341043,21.0,0.299181,0.312684,0.028359
9,30,0.310412,0.342794,34.0,0.299296,0.335803,0.006991


In [17]:
talkingdata_cleaned

,q_i,x_i,public_i,q_j,x_j,public_j,Z
t,,,,,,,
1,10.0,0.977614,0.976513,15,0.981301,0.980518,-0.004005
2,3.0,0.969291,0.968447,17,0.982003,0.981387,-0.012940
3,13.0,0.980837,0.978835,12,0.982868,0.982059,-0.003224
4,20.0,0.983099,0.982106,24,0.983746,0.982751,-0.000645
5,20.0,0.983544,0.982481,16,0.983758,0.982788,-0.000307
6,18.0,0.984053,0.983228,19,0.983677,0.983011,0.000216
7,22.0,0.984332,0.983451,20,0.984126,0.983345,0.000106


In [18]:
landmark_cleaned

,q_i,x_i,public_i,q_j,x_j,public_j,Z
t,,,,,,,
1,4,0.281322,0.278452,12,0.395626,0.400666,-0.122214
2,3,0.210092,0.200356,7,0.404485,0.424232,-0.223876
3,9,0.260463,0.243722,3,0.414758,0.439932,-0.196209
4,17,0.449763,0.453174,20,0.427356,0.451745,0.001429
5,3,0.452996,0.463165,27,0.440531,0.463129,0.000036
6,27,0.476806,0.495764,33,0.435723,0.440375,0.055389
7,20,0.512726,0.533616,20,0.494300,0.517133,0.016482


## Dev Testing

### Data Manipulation Testing for Paper Requirements

In [11]:
vinbigdata_filtered

,SubmissionDate,Public,Private,TeamId,TeamName
8142428,2021-01-04,0.164498,0.159467,6087561,SZI
8142429,2021-01-06,0.124063,0.146764,6087561,SZI
8142430,2021-01-06,0.117363,0.133969,6087561,SZI
8328435,2021-01-07,0.172138,0.167738,6087561,SZI
8328436,2021-01-07,0.160037,0.164868,6087561,SZI
...,...,...,...,...,...
8719802,2021-03-30,0.316657,0.291157,6090014,scumed
8684315,2021-03-30,0.296349,0.305256,6087561,SZI
8684313,2021-03-30,0.287484,0.299893,6087561,SZI
8684314,2021-03-30,0.296358,0.304209,6087561,SZI


In [12]:
# Old Version

# vinbigdata_filtered['t'] = vinbigdata_filtered.SubmissionDate.dt.week
# week0 = vinbigdata_filtered['t'].min() - 1
# vinbigdata_filtered['t'] = vinbigdata_filtered['t'] - week0
# print(vinbigdata_filtered.t.max())
# print(vinbigdata_filtered.t.min())

13
1


C:\Users\z9906\AppData\Local\Temp\ipykernel_5240\966162957.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  vinbigdata_filtered['t'] = vinbigdata_filtered.SubmissionDate.dt.week


In [71]:
start = vinbigdata_filtered.SubmissionDate.min()
vinbigdata_filtered['t'] = (vinbigdata_filtered.SubmissionDate - start).dt.days // 7 + 1
print(vinbigdata_filtered.t.max())
print(vinbigdata_filtered.t.min())

12
1


In [115]:
start = nfl_filtered.groupby('TeamId')['SubmissionDate'].min().max()
nfl_filtered['t'] = (nfl_filtered.SubmissionDate - start).dt.days // 7 + 1
print(nfl_filtered.t.max())
print(nfl_filtered.t.min())

10
-1


In [116]:
nfl_filtered

,SubmissionDate,Public,Private,TeamId,TeamName,t
9561098,2021-08-14,0.302906,0.266706,7243287,tito,-1
9561102,2021-08-14,0.351788,0.265026,7243287,tito,-1
9561099,2021-08-14,0.302955,0.263226,7243287,tito,-1
9561100,2021-08-14,0.347214,0.263172,7243287,tito,-1
9561101,2021-08-14,0.353413,0.269113,7243287,tito,-1
...,...,...,...,...,...,...
9701207,2021-11-02,0.939928,0.947134,7279260,K_mat,10
9695486,2021-11-02,0.935111,0.900798,7243287,tito,10
9695487,2021-11-02,0.933577,0.902260,7243287,tito,10
9695488,2021-11-02,0.927135,0.900704,7243287,tito,10


In [117]:
q_x = nfl_filtered.groupby(by=['TeamId', 't']).agg(q =pd.NamedAgg('SubmissionDate', aggfunc='count'),
                                       x = pd.NamedAgg('Private', aggfunc='max'),
                                       public = pd.NamedAgg('Public', aggfunc='max'))
# q_x.reset_index().sort_values(['TeamId', 't'])
# q_x.reset_index().sort_values(['TeamId','t'])

In [145]:
q_x

q         x    public
TeamId  t                          
7243287 -1    5  0.269113  0.353413
         0   10  0.629647  0.710539
         7   10  0.876757  0.860337
         9    5  0.905786  0.924171
         10  20  0.915345  0.935250
7279260  1    5  0.668100  0.743317
         2    2  0.840817  0.847551
         3    4  0.904971  0.879856
         4    6  0.894157  0.888637
         6    1  0.886581  0.923123
         8    8  0.930886  0.932258
         9    5  0.938919  0.929044
         10  18  0.956714  0.939928

In [131]:
id_list = np.unique([x[0] for x in q_x.index])
df_i = q_x.loc[id_list[0]]
df_j = q_x.loc[id_list[1]]
df_full = df_i.merge(df_j,left_index=True,right_index=True,how='outer', suffixes=('_i','_j'))
df_full[['q_i', 'q_j']] = df_full[['q_i', 'q_j']].fillna(0)
df_full[['x_i', 'x_j', 'public_i', 'public_j']] = df_full[['x_i', 'x_j', 'public_i', 'public_j']].fillna(method='ffill')
df_full[df_full.index > 0]

,q_i,x_i,public_i,q_j,x_j,public_j
t,,,,,,
1,0.0,0.629647,0.710539,5.0,0.668100,0.743317
2,0.0,0.629647,0.710539,2.0,0.840817,0.847551
3,0.0,0.629647,0.710539,4.0,0.904971,0.879856
4,0.0,0.629647,0.710539,6.0,0.894157,0.888637
6,0.0,0.629647,0.710539,1.0,0.886581,0.923123
7,10.0,0.876757,0.860337,0.0,0.886581,0.923123
8,0.0,0.876757,0.860337,8.0,0.930886,0.932258
9,5.0,0.905786,0.924171,5.0,0.938919,0.929044
10,20.0,0.915345,0.935250,18.0,0.956714,0.939928


In [138]:
# Old Version

# id_list = np.unique([x[0] for x in q_x.index])

# q_i = q_x.loc[id_list[0]]['q']
# q_j = q_x.loc[id_list[1]]['q']

# x_i = q_x.loc[id_list[0]]['x']
# x_j = q_x.loc[id_list[1]]['x']

# y_i = x_i - x_j
# y_j = x_j - x_i

# Z = q_x.loc[id_list[0]]['public'] - q_x.loc[id_list[1]]['public']
# t = np.arange(q_i.size) + 1
# # t = t.astype('int64')
# cols = ['q_i', 'q_j', 'x_i', 'x_j', 'y_i', 'y_j', 'Z']

# cleaned_data = pd.DataFrame(data=np.array([q_i, q_j, x_i, x_j, y_i, y_j, Z]).T, columns=cols)
# cleaned_data['t'] = t
# cleaned_data

In [15]:
cleaned_data.to_csv('vinbigdata_cleaned.csv', index=False)

#### Diff data (can be done in R as well, try to keep raw data at this stage)

In [84]:
nabla_cols = cols[2:]
nabla_names = ['diff_' + x for x in nabla_cols]

In [85]:
nabla_df = cleaned_data[nabla_cols].diff().dropna()
nabla_df.columns = nabla_names
nabla_df

,diff_x_i,diff_x_j,diff_y_i,diff_y_j,diff_Z
1,0.020206,0.124464,-0.104258,0.104258,-0.140220
2,0.080207,0.007786,0.072420,-0.072420,0.093993
3,0.002932,0.038886,-0.035954,0.035954,-0.016218
4,0.000156,0.004600,-0.004444,0.004444,0.009590
5,0.006950,-0.007277,0.014228,-0.014228,0.002122
6,0.004559,-0.011765,0.016324,-0.016324,0.005637
7,0.003530,0.010605,-0.007075,0.007075,0.057289
8,0.005535,0.034898,-0.029363,0.029363,-0.064606
9,0.000261,-0.000608,0.000869,-0.000869,-0.018112
10,-0.001042,0.006851,-0.007893,0.007893,0.003894


### For Loop Dev Testing

In [91]:
key_col = 'TeamId'
last_day = ranzcr['SubmissionDate'].max()

ranzcr_names = ranzcr[['TeamId', 'TeamName']].drop_duplicates().set_index('TeamId')
ranzcr_score = ranzcr[ranzcr['SubmissionDate'] == last_day].groupby(key_col)['Private'].max().sort_values(ascending=False)

In [157]:
ranzcr_pub_var = ranzcr.groupby(key_col)['Public'].var()
ranzcr_pub_var.name = 'PublicVar'

In [92]:
ranzcr_submission = ranzcr.groupby(key_col)['SubmissionDate'].count().sort_values(ascending=False)
ranzcr_submission.name = 'Submissions'

In [93]:
ranzcr_first_day = ranzcr.groupby(key_col)['SubmissionDate'].min()
ranzcr_first_day.name = 'FirstDay'

ranzcr_last_day = ranzcr.groupby(key_col)['SubmissionDate'].max()
ranzcr_last_day.name = 'LastDay'

ranzcr_duration = ranzcr_last_day - ranzcr_first_day
ranzcr_duration.name = 'Duration'

In [158]:
ranzcr_summary = pd.concat([ranzcr_names, ranzcr_score, ranzcr_pub_var, ranzcr_duration, ranzcr_first_day, ranzcr_last_day, ranzcr_submission],axis=1).sort_values(['Private', 'Duration', 'Submissions'], ascending=False)
# ranzcr_summary

In [159]:
ranzcr_summary

,TeamName,Private,PublicVar,Duration,FirstDay,LastDay,Submissions
TeamId,,,,,,,
6013849,All Data Are Ext,0.976732,0.000584,48 days,2021-01-27,2021-03-16,22
6012213,リーマン面を這う,0.976420,0.002772,90 days,2020-12-16,2021-03-16,171
6013117,Preferred CLiP,0.976248,0.000056,90 days,2020-12-16,2021-03-16,119
6125018,Watercooled,0.975640,0.009351,59 days,2021-01-16,2021-03-16,116
6012303,Guanshuo Xu,0.975635,0.001052,48 days,2021-01-27,2021-03-16,28
6052677,DatNT,0.975137,0.000011,76 days,2020-12-30,2021-03-16,125
6012271,RaddbotnaKama 200d,0.975069,0.000176,91 days,2020-12-15,2021-03-16,200
6015339,bestfitting,0.974953,0.000335,47 days,2021-01-28,2021-03-16,139
6233219,toxu,0.974553,0.002803,20 days,2021-02-24,2021-03-16,110


### Filtered Index Testing

In [7]:
ranzcr_filtered_team = ranzcr_summary.index[:2]
useful_cols = ['SubmissionDate',
               'Public',
               'Private',
               'TeamId',
               'TeamName']

In [8]:
ranzcr_filtered_data = ranzcr[ranzcr['TeamId'].isin(ranzcr_filtered_team)][useful_cols]

In [18]:
ranzcr_filtered_data.groupby('TeamId')['SubmissionDate'].min()

TeamId
6012213   2020-12-16
6013849   2021-01-27
Name: SubmissionDate, dtype: datetime64[ns]

In [21]:
max_date = ranzcr_filtered_data.groupby('TeamId')['SubmissionDate'].min().max()
max_date

Timestamp('2021-01-27 00:00:00')

In [25]:
ranzcr_filtered_data = ranzcr_filtered_data[ranzcr_filtered_data['SubmissionDate'] >= max_date]

In [27]:
ranzcr_filtered_data.SubmissionDate.min()

Timestamp('2021-01-27 00:00:00')

In [32]:
start = ranzcr_filtered_data.SubmissionDate.min()
t = (ranzcr_filtered_data.SubmissionDate - start ).dt.days // 7 + 1
t

8451022    1
8451023    1
8451024    1
8451025    3
8445528    3
          ..
8859466    7
8859467    7
8859457    7
8859458    7
8859459    7
Name: SubmissionDate, Length: 140, dtype: int64